In [1]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    average_precision_score, roc_auc_score, precision_recall_curve,
    f1_score, precision_score, recall_score, confusion_matrix, classification_report,
    accuracy_score
    )
from sklearn.preprocessing import label_binarize
import time
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
from xgboost import XGBClassifier
import shap
import os
from datetime import datetime
from joblib import dump, load
import warnings
from sklearn.exceptions import FitFailedWarning
import xgboost as xgb
import optuna

c:\Users\ayxbhg\.venvs\py313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def read_from_azure_in_chunks(chunksize=100000):
    try:
        # Set up connection
        cnxn = pyodbc.connect(
            'Driver={ODBC Driver 17 for SQL Server};'
            'Server=XXXXXXXXXXXX;'
            'Authentication=ActiveDirectoryPassword;'
            'Database=XXXXXXXXXXXX;'
            'UID=xxxxxxxxx;'
            'PWD=xxxxxxxxx;'
        )
        print("Connection successful!")

        # Define SQL query
        sql_query = """
        SELECT
            *
        FROM
            pats.vw_ClaimsDenialPrediction
        WHERE
            LIATPCLIid IS NOT NULL            
        """

        # Load data in chunks and concatenate
        chunk_generator = pd.read_sql(sql_query, cnxn, chunksize=chunksize)
        chunks = []
        for i, chunk in enumerate(chunk_generator):
            print(f"Loaded chunk {i+1} with {len(chunk)} rows")
            chunks.append(chunk)

        full_df = pd.concat(chunks, ignore_index=True)
        print(f"\n Total rows combined: {len(full_df)}")
        return full_df

    except pyodbc.DatabaseError as e:
        print('Database Error:', str(e))
    except pyodbc.Error as e:
        print('Connection Error:', str(e))
    finally:
        try:
            cnxn.close()
        except:
            pass

# Main execution
if __name__ == "__main__":
    df = read_from_azure_in_chunks()

    if df is not None:
        print(f"\nTotal rows returned: {len(df)}\n")
        print("Top 5 rows of the DataFrame:")
        print(df.head())
    else:
        print("No data returned.")

Connection successful!


C:\Users\ayxbhg\AppData\Local\Temp\ipykernel_20900\656024245.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk_generator = pd.read_sql(sql_query, cnxn, chunksize=chunksize)


Loaded chunk 1 with 100000 rows
Loaded chunk 2 with 100000 rows
Loaded chunk 3 with 100000 rows
Loaded chunk 4 with 100000 rows
Loaded chunk 5 with 100000 rows
Loaded chunk 6 with 100000 rows
Loaded chunk 7 with 100000 rows
Loaded chunk 8 with 100000 rows
Loaded chunk 9 with 100000 rows
Loaded chunk 10 with 100000 rows
Loaded chunk 11 with 100000 rows
Loaded chunk 12 with 100000 rows
Loaded chunk 13 with 100000 rows
Loaded chunk 14 with 100000 rows
Loaded chunk 15 with 100000 rows
Loaded chunk 16 with 100000 rows
Loaded chunk 17 with 100000 rows
Loaded chunk 18 with 100000 rows
Loaded chunk 19 with 100000 rows
Loaded chunk 20 with 100000 rows
Loaded chunk 21 with 100000 rows
Loaded chunk 22 with 100000 rows
Loaded chunk 23 with 100000 rows
Loaded chunk 24 with 100000 rows
Loaded chunk 25 with 65920 rows

 Total rows combined: 2465920

Total rows returned: 2465920

Top 5 rows of the DataFrame:
  Clinic  TPCLIID  LIATPCLIid   ServiceDt                     Service  \
0    AHK    11443    

In [3]:
print(f"\nTotal rows before deduplication: {len(df)}")

duplicate_count = df.duplicated().sum()
if duplicate_count == 0:
    print("No duplication detected.")
else:
    print(f"Number of duplicated rows: {duplicate_count}")
    df = df.drop_duplicates()
    print(f"Total rows after deduplication: {len(df)}\n")



Total rows before deduplication: 2465920
No duplication detected.


In [4]:
print(df.isnull().sum()) 

Clinic                           0
TPCLIID                          0
LIATPCLIid                       0
ServiceDt                        0
Service                          0
ClaimID                          0
AmountCharged                    0
CPTCode                          0
ClientID                         0
ClaimBillDate                    0
Payer                            0
Provider                         0
AuthStatus                       0
eligStatus                       0
DenialFlag                       0
lastActDt                        0
cliANSI1                   1587687
cliANSI2                   2351312
TotalPaid                        0
TotalAdj                         0
TotalVoid                        0
CoPay                            0
Deduc                            0
CoIns                            0
CltResp                          0
Balance                          0
MultiFlag                        0
SameDayCli                       0
DaysBetServiceToBill

In [5]:
# -*- coding: utf-8 -*-
"""
Full pipeline (native categoricals, random UNSEEN sampling, Optuna tuning, SHAP):
- Keep `ClientID` (do NOT drop it).
- Use XGBoost **native categoricals** (no one-hot) with `enable_categorical=True`.
- Build **UNSEEN** by random sampling from the remainder after taking the training sample (no date cutoff).
- Engineer compact **date features** and drop raw datetime columns before modeling.
- Optuna tunes on CV PR-AUC (wider search for more trial diversity); choose threshold from OOF.
- Compare **F1/precision/recall** at each trial's OOF-chosen threshold on TEST and UNSEEN.
- Save artifacts; score UNSEEN and Azure data; export **SHAP Top-K** explanations.
"""


# ======================================
# CLEANUP + PREP
# ======================================

# Drop unnecessary columns
cols_to_drop = [
    'cliANSI1', 'cliANSI2', 'TotalPaid', 'TotalAdj', 'TotalVoid','DenialFlag',
    'CoPay', 'Deduc', 'CoIns', 'CltResp', 'ClientResp.', 'Balance', 'tpcliAge'
]
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print(f"Columns dropped: {cols_to_drop}")

# Drop rows with nulls in important columns
before_dropna = len(df)
df = df.dropna(subset=['ServiceDt', 'Service', 'CPTCode'])
after_dropna = len(df)
print(f"Rows dropped due to NULLs : {before_dropna - after_dropna}")
print(f"Total rows after cleaning: {len(df)}")

# Step 1: Drop ID columns (KEEP ClientID)
df = df.drop(columns=['TPCLIID', 'ClaimID', 'LIATPCLIid'], errors='ignore')

# Step 2: Target type
df['MultiFlag'] = df['MultiFlag'].astype('category')


# Ensure datetime dtype for feature engineering
for c in ['ClaimBillDate', 'ServiceDt', 'lastActDt']:
    df[c] = pd.to_datetime(df[c], errors='coerce')

# --------------------------------------
# Build TRAIN sample and random UNSEEN
# --------------------------------------

# Use full df as pool
df_train_pool = df.copy()

# --- Randomly sample up to 1,000,000 rows from training pool ---
n_sample = min(2_000_000, len(df_train_pool))
df_sample = df_train_pool.sample(n=n_sample, random_state=123)
print("Sampled training rows:", df_sample.shape[0])

# --- UNSEEN: sample from remainder (no date logic) ---
remainder = df_train_pool.drop(index=df_sample.index)
n_unseen = min(10_000, len(remainder))
df_unseen = remainder.sample(n=n_unseen, random_state=123).copy()
print("Unseen (sampled) rows:", df_unseen.shape[0])


Columns dropped: ['cliANSI1', 'cliANSI2', 'TotalPaid', 'TotalAdj', 'TotalVoid', 'DenialFlag', 'CoPay', 'Deduc', 'CoIns', 'CltResp', 'ClientResp.', 'Balance', 'tpcliAge']
Rows dropped due to NULLs : 0
Total rows after cleaning: 2465920
Sampled training rows: 2000000
Unseen (sampled) rows: 10000


In [6]:

# (Optional) QC: MultiFlag completeness/validity by ClaimBillDate on the *original* df before dates are dropped
try:
    if 'MultiFlag' not in df.columns:
        raise KeyError("MultiFlag not in df; nothing to QC.")

    # Safe, non-mutating date parsing
    claim_dates = pd.to_datetime(df['ClaimBillDate'], errors='coerce').dt.date

    # Treat NaN or blank strings as missing
    mf_obj = df['MultiFlag'].astype('object')
    blank_mask = mf_obj.isna() | (mf_obj.astype(str).str.strip() == '')

    nan_counts = (
        pd.DataFrame({'ClaimDate': claim_dates, 'is_blank': blank_mask})
          .loc[lambda d: d['is_blank']]
          .groupby('ClaimDate')
          .size()
          .reset_index(name='NaN_MultiFlag_Count')
          .sort_values('NaN_MultiFlag_Count', ascending=False)
    )

    print("NaN/blank counts in MultiFlag by ClaimBillDate (overall):")
    print(nan_counts.head(20))

    # Overall missing rate
    total = len(df)
    missing = int(blank_mask.sum())
    print(f"MultiFlag missing/blank: {missing} / {total} ({missing/total:.2%})")

    # Valid/invalid label check (expected from your case: N, Z, P, F)
    expected = {'N', 'Z', 'P', 'F'}
    valid_mask = ~blank_mask & mf_obj.isin(expected)
    invalid_mask = ~blank_mask & ~mf_obj.isin(expected)

    print("MultiFlag distribution (valid labels only):")
    print(df.loc[valid_mask, 'MultiFlag'].value_counts())

    if invalid_mask.any():
        print("⚠️ Found invalid MultiFlag labels (showing top 10):")
        print(df.loc[invalid_mask, 'MultiFlag'].value_counts().head(10))

except Exception as _e:
    print("[Info] Skipped MultiFlag QC:", _e)


NaN/blank counts in MultiFlag by ClaimBillDate (overall):
Empty DataFrame
Columns: [ClaimDate, NaN_MultiFlag_Count]
Index: []
MultiFlag missing/blank: 0 / 2465920 (0.00%)
MultiFlag distribution (valid labels only):
MultiFlag
N    1587649
Z     594457
P     243209
F      40605
Name: count, dtype: int64


In [7]:
# --- Drop raw datetime columns (no date features) ---
drop_dt = ['ServiceDt', 'ClaimBillDate', 'lastActDt']
df_sample = df_sample.drop(columns=drop_dt, errors='ignore')
df_unseen = df_unseen.drop(columns=drop_dt, errors='ignore')

In [8]:
# ======================================
# Hierarchical XGBoost:  Stage A (N vs Denied)  →  Stage B (Z/P/F)
# F1 floors: Stage A = 0.90, Stage B = 0.80
# ======================================
from pandas.api.types import CategoricalDtype as _CatDType

# (Optional) for Azure scoring at the end
try:
    import pyodbc
except Exception:
    pass

# --- Preconditions
assert 'df_sample' in globals(), "df_sample must exist"
assert 'MultiFlag' in df_sample.columns, "df_sample must have MultiFlag"

# ======================================
# Configuration
# ======================================
EXPECTED_LABELS_FULL = ['N', 'Z', 'P', 'F']     # final label order
DENIED_LABELS = ['Z', 'P', 'F']                 # Stage B labels
f1_floor_A = 0.90
f1_floor_B = 0.80
drop_dt = ['ServiceDt', 'ClaimBillDate', 'lastActDt']  # raw datetimes to drop
# Target share for class 'F' in Stage B training AFTER oversampling.
# Example: 0.40 means ~40% of Stage-B training rows will be 'F'.
F_TARGET_RATIO_B = 0.40   # tune 0.25–0.50 based on validation

# Safety: locate the class index for 'F' in Stage B
F_CLASS_IDX_B = DENIED_LABELS.index('F')  # expected 2
# ======================================
# Helpers
# ======================================
def evaluate_predictions(y_true, y_pred_or_proba, class_names=None, thresholds=None):
    """Supports hard labels (1D int) or proba matrix (n,K) with optional per-class thresholds."""
    y_true = np.asarray(y_true).astype(int)
    arr = np.asarray(y_pred_or_proba)

    # Case 1: already hard labels
    if arr.ndim == 1 and np.issubdtype(arr.dtype, np.integer):
        y_pred = arr

    # Case 2: probability matrix -> hard labels
    elif arr.ndim == 2:
        proba = arr.astype(float)
        n_classes = proba.shape[1]
        if thresholds is None:
            y_pred = proba.argmax(axis=1)
        else:
            thr = np.asarray(thresholds, dtype=float)
            if thr.shape != (n_classes,):
                raise ValueError(f"thresholds must have shape {(n_classes,)}, got {thr.shape}")
            over = proba >= thr                 # (n, K)
            any_over = over.any(axis=1)         # (n,)
            scores_over = np.where(over, proba, -np.inf)
            y_pred = scores_over.argmax(axis=1)
            no_over = ~any_over
            if np.any(no_over):
                y_pred[no_over] = proba[no_over].argmax(axis=1)
    else:
        raise ValueError("Unsupported input for y_pred_or_proba.")

    cm = confusion_matrix(y_true, y_pred)
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "f1_weighted": f1_score(y_true, y_pred, average="weighted", zero_division=0),
    }
    # Optional: macro PR-AUC (OvR) if probabilities were provided; skip here for speed.
    rep = classification_report(
        y_true, y_pred,
        target_names=class_names if class_names is not None else None,
        digits=4, zero_division=0
    )
    return cm, metrics, rep

def print_results(model_name, cm, metrics, clsrep):
    print(f"=== {model_name} RESULTS ===")
    print("Confusion Matrix (rows=true, cols=pred):")
    print(cm)
    print("Metrics:")
    for k, v in metrics.items():
        try:
            print(f"  {k}: {v:.5f}")
        except Exception:
            print(f"  {k}: {v}")
    print("Classification Report:\n", clsrep)

def threshold_for_f1(y_true, proba, f1_floor=0.90, grid=None):
    """Pick smallest threshold meeting F1>=floor; else return best-F1 threshold."""
    y_true = np.asarray(y_true).astype(int)
    proba  = np.asarray(proba).ravel()
    if grid is None:
        grid = np.linspace(0.01, 0.99, 99)
    best_thr, best_f1, chosen = 0.5, -1.0, None
    for thr in grid:
        pred = (proba >= thr).astype(int)
        f1 = f1_score(y_true, pred, zero_division=0)
        if f1 >= f1_floor and chosen is None:
            chosen = float(thr)
        if f1 > best_f1:
            best_f1, best_thr = f1, float(thr)
    return chosen if chosen is not None else best_thr

def ovr_thresholds_for_f1(y_true, proba, f1_floor=0.80, grid=None):
    """Per-class thresholds (OvR) using F1>=floor; else return each class's best-F1 thr."""
    y_true = np.asarray(y_true).astype(int)
    proba  = np.asarray(proba)
    K = proba.shape[1]
    if grid is None:
        grid = np.linspace(0.01, 0.99, 99)
    thrs = np.full(K, 0.5, dtype=float)
    for k in range(K):
        y_bin = (y_true == k).astype(int)
        if y_bin.sum() == 0:
            thrs[k] = 0.5; continue
        pk = proba[:, k]
        best_thr, best_f1, chosen = 0.5, -1.0, None
        for thr in grid:
            pred = (pk >= thr).astype(int)
            f1 = f1_score(y_bin, pred, zero_division=0)
            if f1 >= f1_floor and chosen is None:
                chosen = float(thr)
            if f1 > best_f1:
                best_f1, best_thr = f1, float(thr)
        thrs[k] = chosen if chosen is not None else best_thr
    return thrs


def make_class_weights(y_arr):
    classes, counts = np.unique(y_arr, return_counts=True)
    n, k = len(y_arr), len(classes)
    return {int(c): float(n / (k * cnt)) for c, cnt in zip(classes, counts)}

def align_to_training_native(df_raw: pd.DataFrame, feature_cols: list, cat_dtype_map: dict) -> pd.DataFrame:
    """Align columns and categorical vocab exactly to training spec."""
    dfX = df_raw.copy()
    # Drop targets if present
    for tgt in ('MultiFlag', 'DenialFlag'):
        if tgt in dfX.columns:
            dfX = dfX.drop(columns=[tgt])
    # Add missing features
    for c in feature_cols:
        if c not in dfX.columns:
            if c in cat_dtype_map:
                dfX[c] = pd.Series(pd.Categorical([None]*len(dfX), dtype=_CatDType(cat_dtype_map[c])))
            else:
                dfX[c] = 0
    # Enforce categorical vocab
    for c, cats in cat_dtype_map.items():
        if c in dfX.columns:
            dfX[c] = dfX[c].astype(_CatDType(cats))
    # Reorder
    return dfX[feature_cols]

def oversample_class_to_ratio(X, y, target_class: int, target_ratio: float, random_state: int = 123):
    """
    Duplicate rows of `target_class` with replacement so that
    target_class / (total_after_oversample) ≈ target_ratio.
    Validation/test must NEVER pass through here.

    Parameters
    ----------
    X : pd.DataFrame
    y : array-like of shape (n,)
    target_class : int (e.g., 2 for 'F' when DENIED_LABELS=['Z','P','F'])
    target_ratio : float in (0,1), desired final share of target_class
    random_state : int

    Returns
    -------
    X_os : pd.DataFrame
    y_os : np.ndarray
    """
    y_arr = np.asarray(y).astype(int)
    n = len(y_arr)
    if n == 0:
        return X.copy(), y_arr.copy()

    idx_tc = np.flatnonzero(y_arr == target_class)
    if len(idx_tc) == 0:
        # Nothing to oversample; return unchanged
        return X.copy(), y_arr.copy()

    # Current counts
    n_tc = len(idx_tc)
    n_non = n - n_tc
    # If current ratio already >= target, skip
    curr_ratio = n_tc / max(1, n)
    if curr_ratio >= target_ratio:
        # Shuffle for good measure
        rng = np.random.default_rng(random_state)
        order = rng.permutation(n)
        return X.iloc[order].copy(), y_arr[order].copy()

    # Desired total target count after oversampling:
    # Let F' be final target count; with non-target fixed at n_non:
    # F' / (n_non + F') = target_ratio -> F' = (target_ratio / (1 - target_ratio)) * n_non
    target_count = int(np.ceil((target_ratio / (1.0 - target_ratio)) * n_non))
    add_needed = max(0, target_count - n_tc)
    if add_needed == 0:
        rng = np.random.default_rng(random_state)
        order = rng.permutation(n)
        return X.iloc[order].copy(), y_arr[order].copy()

    rng = np.random.default_rng(random_state)
    add_idx = rng.choice(idx_tc, size=add_needed, replace=True)

    # Concatenate originals + sampled duplicates
    X_os = pd.concat([X, X.iloc[add_idx]], axis=0)
    y_os = np.concatenate([y_arr, y_arr[add_idx]], axis=0)

    # Final shuffle
    order = rng.permutation(len(y_os))
    return X_os.iloc[order].copy(), y_os[order].copy()

# ======================================
# CLEAN + TARGETS
# ======================================
# Keep a working copy, drop raw datetime columns
df_work = df_sample.copy()
df_work = df_work.drop(columns=drop_dt, errors='ignore')

# Clean target
df_work['MultiFlag'] = (
    df_work['MultiFlag'].astype(str).str.strip().replace('', np.nan)
)
df_work = df_work[df_work['MultiFlag'].notna()].copy()
df_work['MultiFlag'] = pd.Categorical(df_work['MultiFlag'], categories=EXPECTED_LABELS_FULL, ordered=False)

# Cast predictor categoricals to 'category'
categorical_cols = [
    c for c in df_work.select_dtypes(include=['object', 'category']).columns
    if c != 'MultiFlag'
]
if 'ClientID' in df_work.columns and 'ClientID' not in categorical_cols:
    categorical_cols.append('ClientID')
for c in categorical_cols:
    df_work[c] = df_work[c].astype('category')

# Build X / y (full multi labels)
X_all = df_work.drop(columns=['MultiFlag'])
y_full_codes = df_work['MultiFlag'].cat.codes  # 0..3 -> ['N','Z','P','F']

class_map_full = dict(enumerate(df_work['MultiFlag'].cat.categories))   # {0:'N',1:'Z',2:'P',3:'F'}
inv_map_full   = {v:k for k,v in class_map_full.items()}
code_N = inv_map_full['N']
class_names_full = [class_map_full[i] for i in range(len(class_map_full))]

# Lock category vocab
cat_dtype_map = {c: list(df_work[c].cat.categories) for c in categorical_cols}

# ======================================
# SPLIT + enforce category vocab
# ======================================
X_train_full, X_test, y_full_train_full, y_full_test = train_test_split(
    X_all, y_full_codes, test_size=0.20, random_state=123, stratify=y_full_codes
)
X_train, X_val, y_full_train, y_full_val = train_test_split(
    X_train_full, y_full_train_full, test_size=0.25, random_state=123, stratify=y_full_train_full
)

def _enforce_cats(dfX):
    for c, cats in cat_dtype_map.items():
        if c in dfX.columns:
            dfX[c] = dfX[c].astype(_CatDType(cats))
    return dfX

X_train = _enforce_cats(X_train)
X_val   = _enforce_cats(X_val)
X_test  = _enforce_cats(X_test)

# Show class distributions
def _show_dist(name, y_arr, cmap):
    vc = pd.Series(y_arr).value_counts().sort_index()
    print(f"{name}:", {cmap[int(k)]: int(v) for k, v in vc.items()})

_show_dist("Train full labels", y_full_train, class_map_full)
_show_dist("Val full labels",   y_full_val,   class_map_full)
_show_dist("Test full labels",  y_full_test,  class_map_full)

# ======================================
# Derive Stage A (binary) and Stage B (multi denied)
# ======================================
# Stage A: Denied vs NotDenied
yA_train = (y_full_train != code_N).astype(int)
yA_val   = (y_full_val   != code_N).astype(int)
yA_test  = (y_full_test  != code_N).astype(int)

# Stage B: only denied rows; labels = Z,P,F -> codes 0..2 in DENIED_LABELS order
maskA_train_den = (yA_train == 1)
maskA_val_den   = (yA_val   == 1)
maskA_test_den  = (yA_test  == 1)

yB_train_labels = pd.Series(y_full_train, index=X_train.index).map(lambda c: class_map_full[int(c)]).loc[maskA_train_den]
yB_val_labels   = pd.Series(y_full_val,   index=X_val.index).map(lambda c: class_map_full[int(c)]).loc[maskA_val_den]
yB_test_labels  = pd.Series(y_full_test,  index=X_test.index).map(lambda c: class_map_full[int(c)]).loc[maskA_test_den]

yB_train = pd.Categorical(yB_train_labels, categories=DENIED_LABELS).codes
yB_val   = pd.Categorical(yB_val_labels,   categories=DENIED_LABELS).codes
yB_test  = pd.Categorical(yB_test_labels,  categories=DENIED_LABELS).codes

X_train_B = X_train.loc[maskA_train_den].copy()
X_val_B   = X_val.loc[maskA_val_den].copy()
X_test_B  = X_test.loc[maskA_test_den].copy()

# Assemble tr+val pools
X_trval_A = pd.concat([X_train, X_val], axis=0)
y_trval_A = pd.concat([pd.Series(yA_train, index=X_train.index),
                       pd.Series(yA_val,   index=X_val.index)], axis=0)
X_trval_A = _enforce_cats(X_trval_A)

X_trval_B = pd.concat([X_train_B, X_val_B], axis=0)
y_trval_B = pd.concat([pd.Series(yB_train, index=X_train_B.index),
                       pd.Series(yB_val,   index=X_val_B.index)], axis=0)
X_trval_B = _enforce_cats(X_trval_B)

# Class/sample weights
cw_A  = make_class_weights(yA_train)
sw_A_train = pd.Series(yA_train).map(cw_A).to_numpy()
sw_A_val   = pd.Series(yA_val).map(cw_A).to_numpy()
sw_A_trval = pd.Series(y_trval_A).map(cw_A).to_numpy()

cw_B  = make_class_weights(yB_train)
sw_B_train = pd.Series(yB_train).map(cw_B).to_numpy()
sw_B_val   = pd.Series(yB_val).map(cw_B).to_numpy()
sw_B_trval = pd.Series(y_trval_B).map(cw_B).to_numpy()

print("Stage A class weights:", cw_A)
print("Stage B class weights:", {DENIED_LABELS[k]: v for k, v in cw_B.items()})


Train full labels: {'N': 772630, 'Z': 289384, 'P': 118193, 'F': 19793}
Val full labels: {'N': 257543, 'Z': 96462, 'P': 39398, 'F': 6597}
Test full labels: {'N': 257543, 'Z': 96461, 'P': 39398, 'F': 6598}
Stage A class weights: {0: 0.7765683444857177, 1: 1.4039356997449517}
Stage B class weights: {'Z': 0.492275546217713, 'P': 1.2052885252651737, 'F': 7.197325653850688}


In [9]:

# ======================================
# Stage A (Binary) — Optuna
# ======================================
CV_A = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
def split_A():
    return CV_A.split(X_trval_A, y_trval_A)

def objective_A(trial):
    params = {
        "tree_method": "hist",
        "objective": "binary:logistic",
        "eval_metric": "aucpr",
        "eta": trial.suggest_float("eta", 0.02, 0.20, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 20.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-6, 50.0, log=True),
        "reg_alpha":  trial.suggest_float("reg_alpha",  1e-6, 20.0, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 10.0),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "max_bin": trial.suggest_categorical("max_bin", [64, 128, 256]),
        "max_cat_to_onehot": trial.suggest_categorical("max_cat_to_onehot", [1, 2, 10]),
        "sampling_method": "uniform",
        "seed": 123,
    }
    f1s = []
    for tr_idx, va_idx in split_A():
        dtr = xgb.DMatrix(X_trval_A.iloc[tr_idx], label=y_trval_A.iloc[tr_idx],
                          weight=sw_A_trval[tr_idx], enable_categorical=True)
        dva = xgb.DMatrix(X_trval_A.iloc[va_idx], label=y_trval_A.iloc[va_idx],
                          weight=sw_A_trval[va_idx], enable_categorical=True)
        booster = xgb.train(params, dtr, num_boost_round=4000, evals=[(dva,"val")],
                            early_stopping_rounds=150, verbose_eval=False)
        proba = booster.predict(dva, iteration_range=(0, booster.best_iteration + 1))
        # Max F1 per fold (binary)
        grid = np.linspace(0.05, 0.95, 19)
        best_f1 = -1.0
        for thr in grid:
            pred = (proba >= thr).astype(int)
            best_f1 = max(best_f1, f1_score(y_trval_A.iloc[va_idx], pred, zero_division=0))
        f1s.append(best_f1)
    return float(np.mean(f1s))

study_A = optuna.create_study(direction="maximize")
study_A.optimize(objective_A, n_trials=30, show_progress_bar=False)
best_params_A = study_A.best_params.copy()
best_params_A.update({
    "tree_method":"hist","objective":"binary:logistic","eval_metric":"aucpr","seed":123
})

# OOF to get F1-floor threshold and best n_estimators
def get_oof_threshold_and_n_binary(X_ref, y_ref, params, f1_floor=0.90, weights=None):
    y_ref = np.asarray(y_ref).astype(int)
    oof_proba = np.full(shape=(len(y_ref),), fill_value=np.nan, dtype=float)
    best_iters = []
    for tr_idx, va_idx in split_A():
        dtr = xgb.DMatrix(X_ref.iloc[tr_idx], label=y_ref[tr_idx],
                          weight=None if weights is None else np.asarray(weights)[tr_idx],
                          enable_categorical=True)
        dva = xgb.DMatrix(X_ref.iloc[va_idx], label=y_ref[va_idx],
                          weight=None if weights is None else np.asarray(weights)[va_idx],
                          enable_categorical=True)
        booster = xgb.train(params, dtr, num_boost_round=4000, evals=[(dva,"val")],
                            early_stopping_rounds=150, verbose_eval=False)
        proba = booster.predict(dva, iteration_range=(0, booster.best_iteration + 1))
        oof_proba[va_idx] = proba
        best_iters.append(int(booster.best_iteration) + 1)
    thr = threshold_for_f1(y_ref, oof_proba, f1_floor=f1_floor)
    return float(thr), int(np.median(best_iters))

thr_A, nA = get_oof_threshold_and_n_binary(X_trval_A, y_trval_A, best_params_A,
                                           f1_floor=f1_floor_A, weights=sw_A_trval)

# Refit Stage A
xgbA = XGBClassifier(
    tree_method="hist", objective="binary:logistic",
    n_estimators=nA,
    learning_rate=best_params_A.get("eta", 0.06),
    max_depth=best_params_A.get("max_depth", 8),
    min_child_weight=best_params_A.get("min_child_weight", 2.0),
    subsample=best_params_A.get("subsample", 0.9),
    colsample_bytree=best_params_A.get("colsample_bytree", 0.8),
    reg_lambda=best_params_A.get("reg_lambda", 1.0),
    reg_alpha=best_params_A.get("reg_alpha", 0.0),
    gamma=best_params_A.get("gamma", 0.0),
    grow_policy=best_params_A.get("grow_policy", "depthwise"),
    max_bin=best_params_A.get("max_bin", 256),
    max_cat_to_onehot=best_params_A.get("max_cat_to_onehot", 1),
    sampling_method="uniform",
    random_state=123, n_jobs=-1, enable_categorical=True
)
xgbA.fit(X_trval_A, y_trval_A, sample_weight=sw_A_trval)

# ======================================
# Stage B (Multi: Z/P/F) — Optuna
# ======================================
CV_B = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
def split_B():
    return CV_B.split(X_trval_B, y_trval_B)

def objective_B(trial):
    params = {
        "tree_method": "hist",
        "objective": "multi:softprob",
        "num_class": 3,
        "eval_metric": ["mlogloss", "merror"],
        "eta": trial.suggest_float("eta", 0.02, 0.20, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 20.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-6, 50.0, log=True),
        "reg_alpha":  trial.suggest_float("reg_alpha",  1e-6, 20.0, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 10.0),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "max_bin": trial.suggest_categorical("max_bin", [64, 128, 256]),
        "max_cat_to_onehot": trial.suggest_categorical("max_cat_to_onehot", [1, 2, 10]),
        "sampling_method": "uniform",
        "seed": 123,
    }
    f1s = []
    for tr_idx, va_idx in split_B():
        # Build train/val splits
        X_tr = X_trval_B.iloc[tr_idx]
        y_tr = y_trval_B.iloc[tr_idx].to_numpy()
        X_va = X_trval_B.iloc[va_idx]
        y_va = y_trval_B.iloc[va_idx].to_numpy()

        # --- Oversample class F on TRAIN ONLY ---
        X_tr_os, y_tr_os = oversample_class_to_ratio(
            X_tr, y_tr, target_class=F_CLASS_IDX_B, target_ratio=F_TARGET_RATIO_B, random_state=123
        )

        # DMatrix with NO weights (oversampling already shifts priors)
        dtr = xgb.DMatrix(X_tr_os, label=y_tr_os, enable_categorical=True)
        dva = xgb.DMatrix(X_va,     label=y_va,     enable_categorical=True)

        booster = xgb.train(
            params, dtr, num_boost_round=4000, evals=[(dva, "val")],
            early_stopping_rounds=150, verbose_eval=False
        )
        proba = booster.predict(dva, iteration_range=(0, booster.best_iteration + 1))
        y_pred = proba.argmax(axis=1)
        f1s.append(f1_score(y_va, y_pred, average="macro", zero_division=0))
    return float(np.mean(f1s))

study_B = optuna.create_study(direction="maximize")
study_B.optimize(objective_B, n_trials=30, show_progress_bar=False)
best_params_B = study_B.best_params.copy()
best_params_B.update({
    "tree_method":"hist","objective":"multi:softprob",
    "num_class":3,"eval_metric":["mlogloss","merror"],"seed":123
})

def get_oof_thresholds_and_n_multi(X_ref, y_ref, params, f1_floor=0.80, weights=None):
    y_ref = np.asarray(y_ref).astype(int)
    K = int(len(np.unique(y_ref)))
    oof_proba = np.full(shape=(len(y_ref), K), fill_value=np.nan, dtype=float)
    best_iters = []
    for tr_idx, va_idx in split_B():
        X_tr = X_ref.iloc[tr_idx]
        y_tr = y_ref[tr_idx]
        X_va = X_ref.iloc[va_idx]
        y_va = y_ref[va_idx]

        # --- Oversample F on TRAIN ONLY ---
        X_tr_os, y_tr_os = oversample_class_to_ratio(
            X_tr, y_tr, target_class=F_CLASS_IDX_B, target_ratio=F_TARGET_RATIO_B, random_state=123
        )

        dtr = xgb.DMatrix(X_tr_os, label=y_tr_os, enable_categorical=True)
        dva = xgb.DMatrix(X_va,     label=y_va,     enable_categorical=True)

        booster = xgb.train(
            params, dtr, num_boost_round=4000, evals=[(dva, "val")],
            early_stopping_rounds=150, verbose_eval=False
        )
        proba = booster.predict(dva, iteration_range=(0, booster.best_iteration + 1))
        oof_proba[va_idx, :] = proba
        best_iters.append(int(booster.best_iteration) + 1)

    thr_vec = ovr_thresholds_for_f1(y_ref, oof_proba, f1_floor=f1_floor)
    return thr_vec, int(np.median(best_iters))

# ======================================
# Compute Stage B thresholds / n_estimators (CHANGE THE CALL to avoid passing weights)
# ======================================
thr_vec_B, nB = get_oof_thresholds_and_n_multi(
    X_trval_B, y_trval_B, best_params_B, f1_floor=f1_floor_B, weights=None  # weights unused with oversampling
)


# ======================================
# Refit Stage B (REPLACE this block to oversample on the full train+val pool)
# ======================================
# Oversample 'F' on the full Stage-B training pool before final fit
X_trval_B_os, y_trval_B_os = oversample_class_to_ratio(
    X_trval_B, y_trval_B.to_numpy(), target_class=F_CLASS_IDX_B,
    target_ratio=F_TARGET_RATIO_B, random_state=123
)

xgbB = XGBClassifier(
    tree_method="hist", objective="multi:softprob", num_class=3,
    n_estimators=nB,
    learning_rate=best_params_B.get("eta", 0.06),
    max_depth=best_params_B.get("max_depth", 8),
    min_child_weight=best_params_B.get("min_child_weight", 2.0),
    subsample=best_params_B.get("subsample", 0.9),
    colsample_bytree=best_params_B.get("colsample_bytree", 0.8),
    reg_lambda=best_params_B.get("reg_lambda", best_params_B.get("lambda", 1.0)),
    reg_alpha=best_params_B.get("reg_alpha", 0.0),
    gamma=best_params_B.get("gamma", 0.0),
    grow_policy=best_params_B.get("grow_policy", "depthwise"),
    max_bin=best_params_B.get("max_bin", 256),
    max_cat_to_onehot=best_params_B.get("max_cat_to_onehot", 1),
    sampling_method="uniform",
    random_state=123, n_jobs=-1, enable_categorical=True
)
# No sample_weight here — we already oversampled
xgbB.fit(X_trval_B_os, y_trval_B_os)

# ======================================
# Compose FINAL predictions on TEST
# ======================================
# Stage A decision
probaA_test = xgbA.predict_proba(X_test)[:, 1]
predA_test  = (probaA_test >= thr_A).astype(int)  # 1 = Denied, 0 = NotDenied

# Stage B (for all; use only where predA=1)
probaB_test = xgbB.predict_proba(X_test)         # n x 3
thrB = np.asarray(thr_vec_B, dtype=float)
overB = probaB_test >= thrB
any_overB = overB.any(axis=1)
scores_overB = np.where(overB, probaB_test, -np.inf)
predB_all = scores_overB.argmax(axis=1)
predB_all[~any_overB] = probaB_test[~any_overB].argmax(axis=1)

# Merge to full labels (codes)
pred_full_codes = np.full(shape=(len(X_test),), fill_value=code_N, dtype=int)  # default 'N'
map_B_to_full = {i: inv_map_full[DENIED_LABELS[i]] for i in range(3)}
denied_idx = np.where(predA_test == 1)[0]
pred_full_codes[denied_idx] = np.vectorize(map_B_to_full.get)(predB_all[denied_idx])

# Evaluate
cmT, metT, repT = evaluate_predictions(y_full_test, pred_full_codes, class_names=class_names_full)
print_results("HIERARCHICAL — TEST", cmT, metT, repT)
 

[I 2025-09-13 20:20:55,211] A new study created in memory with name: no-name-1d0fb7d3-c406-4f42-a4f3-cc0c07f71eca
[I 2025-09-13 20:44:00,805] Trial 0 finished with value: 0.9114492410846406 and parameters: {'eta': 0.14489833359973686, 'max_depth': 8, 'min_child_weight': 2.115927435351363, 'subsample': 0.9316564160887764, 'colsample_bytree': 0.7160678146582435, 'reg_lambda': 12.344998218176437, 'reg_alpha': 0.019389406313206712, 'gamma': 2.147518189419446, 'grow_policy': 'lossguide', 'max_bin': 64, 'max_cat_to_onehot': 10}. Best is trial 0 with value: 0.9114492410846406.
[I 2025-09-13 21:08:08,965] Trial 1 finished with value: 0.9097811385648663 and parameters: {'eta': 0.19319358912594076, 'max_depth': 7, 'min_child_weight': 1.677834150039486, 'subsample': 0.502371283293334, 'colsample_bytree': 0.732963601126652, 'reg_lambda': 42.13895959885411, 'reg_alpha': 1.2904163240330186, 'gamma': 1.2564689356103675, 'grow_policy': 'depthwise', 'max_bin': 128, 'max_cat_to_onehot': 1}. Best is tria

=== HIERARCHICAL — TEST RESULTS ===
Confusion Matrix (rows=true, cols=pred):
[[238580  16612   1035   1316]
 [  7036  86228   1454   1743]
 [  1902   2888  34303    305]
 [   727    419    168   5284]]
Metrics:
  accuracy: 0.91099
  f1_macro: 0.84656
  f1_weighted: 0.91261
Classification Report:
               precision    recall  f1-score   support

           N     0.9611    0.9264    0.9434    257543
           Z     0.8123    0.8939    0.8512     96461
           P     0.9281    0.8707    0.8985     39398
           F     0.6110    0.8008    0.6932      6598

    accuracy                         0.9110    400000
   macro avg     0.8281    0.8730    0.8466    400000
weighted avg     0.9162    0.9110    0.9126    400000



In [10]:

# ======================================
# Save artifacts
# ======================================
feature_cols = X_all.columns.tolist()
final_artifacts = {
    "stageA_model": xgbA,
    "stageA_threshold": float(thr_A),
    "stageB_model": xgbB,
    "stageB_thresholds": [float(x) for x in thr_vec_B],
    "feature_cols": feature_cols,
    "categorical_cols": categorical_cols,
    "cat_dtypes": cat_dtype_map,
    "class_names_full": class_names_full,
    "class_names_B": DENIED_LABELS,
    "params_A": best_params_A,
    "params_B": best_params_B,
}
# Format current datetime as YYYYMMDD_HHMMSS
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"xgb_claims_multicat_model_2stage_{current_time}.joblib"
dump(final_artifacts, filename)
print(f"Saved: {filename}")


Saved: xgb_claims_multicat_model_2stage_20250914_180851.joblib


In [12]:
# ======================================
# PREDICT ON df_unseen (if provided)
# ======================================

# Preconditions (must be defined earlier in your script)
assert 'drop_dt' in globals()
assert 'align_to_training_native' in globals()
assert 'feature_cols' in globals()
assert 'cat_dtype_map' in globals()
assert 'final_artifacts' in globals()
assert 'DENIED_LABELS' in globals()          # e.g., ['Z','P','F']
assert 'class_names_full' in globals()       # e.g., ['N','Z','P','F'] (order matters)
assert 'map_B_to_full' in globals()          # maps Stage B idx {0,1,2} -> int code in class_names_full
assert 'code_N' in globals()                 # int code for 'N' in class_names_full (typically 0)

if 'df_unseen' in globals():
    # --- Align to training schema ---
    df_unseen_sc = df_unseen.copy()
    df_unseen_sc = df_unseen_sc.drop(columns=drop_dt, errors='ignore')
    X_unseen = align_to_training_native(df_unseen_sc, feature_cols, cat_dtype_map)

    # ---------- Stage A (N vs Denied) ----------
    pA = final_artifacts["stageA_model"].predict_proba(X_unseen)[:, 1]
    denied_pred = (pA >= final_artifacts["stageA_threshold"]).astype(int)  # numpy array

    # ---------- Stage B (Z/P/F) ----------
    pB = final_artifacts["stageB_model"].predict_proba(X_unseen)            # shape: (n, 3)
    thrB = np.asarray(final_artifacts["stageB_thresholds"], dtype=float)    # shape: (3,)
    overB = pB >= thrB                                                       # broadcast → (n, 3)
    any_overB = overB.any(axis=1)
    scores_overB = np.where(overB, pB, -np.inf)
    predB_all = scores_overB.argmax(axis=1)
    # Fallback to argmax if none of the class probs clear thresholds
    predB_all[~any_overB] = pB[~any_overB].argmax(axis=1)

    # ---------- Compose final 4-class predictions ----------
    pred_full_code = np.full(len(X_unseen), fill_value=code_N, dtype=int)
    idx_denied = np.where(denied_pred == 1)[0]
    # Map Stage B indices (0/1/2) to full-space integer codes for class_names_full
    pred_full_code[idx_denied] = np.vectorize(map_B_to_full.get)(predB_all[idx_denied])

    labels_full = [class_names_full[i] for i in pred_full_code]             # predicted string labels

    # ---------- Package outputs ----------
    results_unseen = df_unseen.copy()
    results_unseen["proba_deny"] = pA
    for i, lbl in enumerate(DENIED_LABELS):
        results_unseen[f"proba_{lbl}"] = pB[:, i]
    results_unseen["xgb_pred_code"] = pred_full_code
    results_unseen["xgb_pred"] = labels_full

    # Attach ground truth if present
    if "MultiFlag" in df_unseen.columns:
        results_unseen["MultiFlag_true"] = df_unseen["MultiFlag"]

    ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    out_pred_fn = f"xgb_unseen_predictions_hier_{ts}.csv"
    results_unseen.to_csv(out_pred_fn, index=False)
    print("Saved UNSEEN predictions:", out_pred_fn)

    # Optional: safe preview without hiding real errors
    try:
        print(results_unseen.head().to_string(index=False))
    except (UnicodeEncodeError, BrokenPipeError) as e:
        print(f"(Skipping preview due to console issue: {e.__class__.__name__})")

    # ---------- Classification reports ----------
    if "MultiFlag" in df_unseen.columns:
        mf = df_unseen["MultiFlag"]

        # Normalize y_true to string labels matching class_names_full
        if pd.api.types.is_numeric_dtype(mf):
            # Treat numbers as codes into class_names_full
            y_true_full = []
            for x in mf.values:
                if pd.isna(x):
                    y_true_full.append(None)
                else:
                    xi = int(x)
                    y_true_full.append(class_names_full[xi] if 0 <= xi < len(class_names_full) else None)
        else:
            y_true_full = mf.astype(str).tolist()

        # Convert to NumPy arrays (FIX for list + NumPy index mismatch)
        y_true_full = np.asarray(y_true_full, dtype=object)
        y_pred_full = np.asarray(labels_full)

        # ---- Full 4-class report (N/Z/P/F) ----
        valid_mask_full = np.isin(y_true_full, class_names_full)
        y_true_full_f = y_true_full[valid_mask_full]
        y_pred_full_f = y_pred_full[valid_mask_full]

        print("\n=== Full 4-class report (N / Z / P / F) ===")
        print(classification_report(
            y_true_full_f, y_pred_full_f,
            labels=class_names_full, digits=4, zero_division=0
        ))

        cm_full = confusion_matrix(y_true_full_f, y_pred_full_f, labels=class_names_full)
        cm_full_df = pd.DataFrame(
            cm_full,
            index=[f"true_{l}" for l in class_names_full],
            columns=[f"pred_{l}" for l in class_names_full]
        )
        cm_full_fn = f"classification_confusion_full_{ts}.csv"
        cm_full_df.to_csv(cm_full_fn, index=True)
        print("Saved 4-class confusion matrix:", cm_full_fn)

        # ---- Stage A (binary: N vs Denied) ----
        y_true_A = np.where(y_true_full == 'N', 0, np.where(np.isin(y_true_full, DENIED_LABELS), 1, -1))
        valid_mask_A = (y_true_A >= 0)
        y_true_A_f = y_true_A[valid_mask_A]
        denied_pred_f = denied_pred[valid_mask_A]

        print("\n=== Stage A report (N vs Denied) ===")
        print(classification_report(
            y_true_A_f, denied_pred_f,
            labels=[0, 1], target_names=['N', 'Denied'],
            digits=4, zero_division=0
        ))

        cm_A = confusion_matrix(y_true_A_f, denied_pred_f, labels=[0, 1])
        cm_A_df = pd.DataFrame(cm_A, index=['true_N', 'true_Denied'], columns=['pred_N', 'pred_Denied'])
        cm_A_fn = f"classification_confusion_stageA_{ts}.csv"
        cm_A_df.to_csv(cm_A_fn, index=True)
        print("Saved Stage A confusion matrix:", cm_A_fn)

        # ---- Stage B (Z/P/F) on rows where truth is Denied AND Stage A predicted Denied ----
        idx_stageB_eval = np.where(np.isin(y_true_full, DENIED_LABELS) & (denied_pred == 1))[0]
        if idx_stageB_eval.size > 0:
            y_true_B = y_true_full[idx_stageB_eval]
            y_pred_B = y_pred_full[idx_stageB_eval]

            print("\n=== Stage B report (Z / P / F) on rows passing Stage A ===")
            print(classification_report(
                y_true_B, y_pred_B,
                labels=DENIED_LABELS, digits=4, zero_division=0
            ))

            cm_B = confusion_matrix(y_true_B, y_pred_B, labels=DENIED_LABELS)
            cm_B_df = pd.DataFrame(
                cm_B,
                index=[f"true_{l}" for l in DENIED_LABELS],
                columns=[f"pred_{l}" for l in DENIED_LABELS]
            )
            cm_B_fn = f"classification_confusion_stageB_{ts}.csv"
            cm_B_df.to_csv(cm_B_fn, index=True)
            print("Saved Stage B confusion matrix:", cm_B_fn)
        else:
            print("\n[Stage B] No rows where truth is Denied and Stage A predicted Denied; skipping Stage B report.")
    else:
        print("\nGround truth ('MultiFlag') not found in df_unseen; classification reports skipped.")


Saved UNSEEN predictions: xgb_unseen_predictions_hier_2025-09-14_18-09-14.csv
Clinic                       Service  AmountCharged CPTCode  ClientID                           Payer   Provider AuthStatus eligStatus MultiFlag  SameDayCli  DaysBetServiceToBilling  proba_deny  proba_Z  proba_P  proba_F  xgb_pred_code xgb_pred MultiFlag_true
  VWBY            Take Home Dose MMT          25.40   H0020       132          HealthPartners PMAP MN 1932375110        N/A        N/A         Z           0                        7    0.999974 0.999569 0.000053 0.000378              1        Z              Z
  B42D         Individual Counseling          96.00   H0004      2156           Anthem HealthKeepers  1922584366        N/A        N/A         N           0                        3    0.039964 0.010896 0.000730 0.988374              0        N              N
    RE E&M New Patient Visit - 99205         144.62   99205      5500             Anthem Medicaid NV  1427042118        N/A        N/A        

In [13]:

# ======================================
# Azure: Predict future claims (Hierarchical)
# ======================================
# Uncomment/configure to run
"""
SQL = '''
SELECT *
FROM pats.vw_ClaimsDenialPrediction
WHERE ClaimBillDate > '2025-08-11' AND LIATPCLIid IS NULL
'''
CNXN_STR = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=bhgazuresql01.database.windows.net;"
    "Authentication=ActiveDirectoryPassword;"
    "Database=xxxxxxx;"
    "UID=xxxxxxx;"
    "PWD=xxxxxxx;"
)

print("Connecting to Azure SQL...")
cnxn = pyodbc.connect(CNXN_STR)
df_raw = pd.read_sql(SQL, cnxn)
cnxn.close()
print(f"Loaded {len(df_raw):,} rows.")

# Keep original columns; drop raw datetime cols as in training
df_out = df_raw.copy()
df_feat = df_raw.drop(columns=drop_dt, errors='ignore')

# Align + predict
X_future = align_to_training_native(df_feat, final_artifacts["feature_cols"], final_artifacts["cat_dtypes"])

# Stage A
pA = final_artifacts["stageA_model"].predict_proba(X_future)[:, 1]
denied_pred = (pA >= final_artifacts["stageA_threshold"]).astype(int)

# Stage B
pB = final_artifacts["stageB_model"].predict_proba(X_future)
thrB = np.asarray(final_artifacts["stageB_thresholds"], dtype=float)
overB = pB >= thrB
any_overB = overB.any(axis=1)
scores_overB = np.where(overB, pB, -np.inf)
predB_all = scores_overB.argmax(axis=1)
predB_all[~any_overB] = pB[~any_overB].argmax(axis=1)

# Compose
pred_full = np.full(shape=(len(X_future),), fill_value=code_N, dtype=int)
idx_d = np.where(denied_pred == 1)[0]
pred_full[idx_d] = np.vectorize(map_B_to_full.get)(predB_all[idx_d])
labels_full = [class_names_full[i] for i in pred_full]

# Attach predictions
for i, lbl in enumerate(DENIED_LABELS):
    df_out[f"proba_{lbl}"] = pB[:, i]
df_out["proba_deny"] = pA
df_out["xgb_pred_code"] = pred_full
df_out["xgb_pred"] = labels_full

print(df_out.head())
print("Predicted class distribution:", df_out["xgb_pred"].value_counts().to_dict())

ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"xgb_prediction_hier_{ts}.csv"
df_out.to_csv(file_name, index=False)
print(f"Saved: {file_name}")
"""


'\nSQL = \'\'\'\nSELECT *\nFROM pats.vw_ClaimsDenialPrediction\nWHERE ClaimBillDate > \'2025-08-11\' AND LIATPCLIid IS NULL\n\'\'\'\nCNXN_STR = (\n    "Driver={ODBC Driver 17 for SQL Server};"\n    "Server=bhgazuresql01.database.windows.net;"\n    "Authentication=ActiveDirectoryPassword;"\n    "Database=xxxxxxx;"\n    "UID=xxxxxxx;"\n    "PWD=xxxxxxx;"\n)\n\nprint("Connecting to Azure SQL...")\ncnxn = pyodbc.connect(CNXN_STR)\ndf_raw = pd.read_sql(SQL, cnxn)\ncnxn.close()\nprint(f"Loaded {len(df_raw):,} rows.")\n\n# Keep original columns; drop raw datetime cols as in training\ndf_out = df_raw.copy()\ndf_feat = df_raw.drop(columns=drop_dt, errors=\'ignore\')\n\n# Align + predict\nX_future = align_to_training_native(df_feat, final_artifacts["feature_cols"], final_artifacts["cat_dtypes"])\n\n# Stage A\npA = final_artifacts["stageA_model"].predict_proba(X_future)[:, 1]\ndenied_pred = (pA >= final_artifacts["stageA_threshold"]).astype(int)\n\n# Stage B\npB = final_artifacts["stageB_model"].